In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
nameCSV = 'innisfail_file.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]

In [4]:
netCDF4.num2date(tt[0:200],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 3, 31, 8, 13, 38, 15623),
       datetime.datetime(1985, 3, 31, 8, 13, 38, 992192),
       datetime.datetime(1985, 3, 31, 8, 13, 39, 976566),
       datetime.datetime(1985, 3, 31, 8, 13, 40, 953124),
       datetime.datetime(1985, 3, 31, 8, 13, 41, 929683),
       datetime.datetime(1985, 3, 31, 8, 13, 42, 914067),
       datetime.datetime(1985, 3, 31, 8, 13, 43, 890626),
       datetime.datetime(1985, 3, 31, 8, 13, 44, 875000),
       datetime.datetime(1985, 3, 31, 8, 13, 45, 851559),
       datetime.datetime(1985, 3, 31, 8, 13, 46, 835933),
       datetime.datetime(1985, 3, 31, 8, 13, 47, 812502),
       datetime.datetime(1985, 3, 31, 8, 13, 48, 789061),
       datetime.datetime(1985, 3, 31, 8, 13, 49, 773435),
       datetime.datetime(1985, 3, 31, 8, 13, 50, 750003),
       datetime.datetime(1985, 3, 31, 8, 13, 51, 734378),
       datetime.datetime(1985, 3, 31, 8, 13, 52, 710936),
       datetime.datetime(1985, 3, 31, 8, 13, 53, 695310),
       datetime

In [38]:
#TEST TO SEE ALL HIGH VALUES
idd = np.where(np.logical_and(wh>22, wh<30))[0]

for k in range(len(idd)):
    print wh[idd[k]], netCDF4.num2date(tt[idd[k]],u'days since 1985-01-01 00:00:00 UTC')

24.345001220703125 2013-02-18 14:34:14.718752
23.207000732421875 2019-02-17 07:54:46.398436
22.24700164794922 2019-02-17 07:54:47.437496
22.306001663208008 2019-02-17 07:54:48.476567
22.29300117492676 2019-02-17 07:54:49.515628
23.23900032043457 2019-02-17 07:54:50.554689
22.00400161743164 2019-02-17 20:19:44.406252


In [11]:
tt1 = netCDF4.num2date(tt,u'days since 1985-01-01 00:00:00 UTC')
print tt1[0], wh[0]
print netCDF4.num2date(max(tt),u'days since 1985-01-01 00:00:00 UTC')

1985-03-31 08:13:38.015623 1.90500009059906
2019-02-20 11:54:42.070312


In [12]:
#MAKE MEAN SATELLITE TRACK. DOESNT SEEM TO WORK??
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [13]:
print netCDF4.num2date(tt[days[0]],u'days since 1985-01-01 00:00:00 UTC')#same as first record

# meanwh is the mean wh of each day (satellite track)
meanwh = []

for k in range(len(days)-1):
    meanwh.append(np.mean(wh[days[k]:days[k+1]])) 
    
print np.mean(meanwh), 'mean from each satellite track'
print np.mean(wh), 'mean from each record'
print len(wh), 'number of records'
print len(meanwh), 'number of satellite tracks'

#slightly different mean wh depending on if done by day, or individual records (each second)

1985-03-31 08:13:38.015623
1.4841182633256176 mean from each satellite track
1.5340450601878108 mean from each record
63418 number of records
2937 number of satellite tracks


In [14]:
netCDF4.num2date(days,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 1, 0, 0), real_datetime(1985, 1, 19, 0, 0),
       real_datetime(1985, 1, 23, 0, 0), ...,
       real_datetime(2158, 5, 28, 0, 0), real_datetime(2158, 6, 22, 0, 0),
       real_datetime(2158, 7, 12, 0, 0)], dtype=object)

In [16]:
len(days)

2938

MEAN MONTHLY VALUES

In [34]:
for k in range (len(meanwh)):
    print netCDF4.num2date(meanwh[k],u'days since 1985-01-01 00:00:00 UTC')

1985-01-03 03:53:07.209320
1985-01-01 06:59:02.401385
1985-01-03 03:56:09.602966
1985-01-02 13:54:54.727020
1985-01-03 07:27:15.851898
1985-01-01 11:35:02.402973
1985-01-02 06:31:21.604385
1985-01-01 11:21:22.910598
1985-01-01 09:51:41.761780
1985-01-03 00:01:26.414337
1985-01-02 05:47:02.404861
1985-01-02 17:33:25.533288
1985-01-02 02:49:16.803131
1985-01-01 13:14:24.002323
1985-01-01 18:26:47.042999
1985-01-01 15:42:02.882538
1985-01-02 01:48:24.689516
1985-01-02 17:52:00.008469
1985-01-03 00:01:31.808152
1985-01-03 07:50:57.355981
1985-01-01 12:26:07.544379
1985-01-03 18:26:15.147893
1985-01-02 13:07:19.205933
1985-01-02 09:02:42.641076
1985-01-03 11:24:27.010403
1985-01-02 15:52:43.206482
1985-01-01 17:43:12.003107
1985-01-01 16:50:52.804470
1985-01-02 11:05:16.806221
1985-01-02 21:28:48.008652
1985-01-02 09:54:00.003777
1985-01-02 15:33:36.008377
1985-01-01 19:47:35.317252
1985-01-02 10:03:12.964211
1985-01-02 23:13:12.007828
1985-01-02 19:12:43.208199
1985-01-02 03:42:28.805809
1

1985-01-03 04:41:57.127075
1985-01-02 05:32:21.124878
1985-01-02 07:06:46.806028
1985-01-02 03:45:56.950222
1985-01-02 12:56:24.005356
1985-01-03 10:44:55.691986
1985-01-01 17:41:51.363716
1985-01-01 18:53:11.403329
1985-01-01 15:54:43.202577
1985-01-02 12:36:37.034127
1985-01-01 15:14:24.002717
1985-01-02 14:31:55.208817
1985-01-01 23:56:40.203538
1985-01-01 22:39:36.002455
1985-01-02 15:31:53.148390
1985-01-02 10:13:48.006721
1985-01-02 17:50:06.007004
1985-01-01 17:44:44.163551
1985-01-02 01:44:13.204465
1985-01-02 10:03:36.004944
1985-01-03 17:56:38.410492
1985-01-03 05:38:35.527267
1985-01-03 05:41:22.210207
1985-01-03 09:26:18.774678
1985-01-01 10:58:30.721664
1985-01-02 17:04:59.528503
1985-01-02 07:44:15.364609
1985-01-03 04:55:46.568413
1985-01-02 05:11:13.925400
1985-01-02 17:20:49.926224
1985-01-02 19:14:06.727295
1985-01-02 23:48:05.245098
1985-01-02 14:18:51.433868
1985-01-01 23:54:25.924644
1985-01-03 03:52:30.730591
1985-01-02 16:53:57.950461
1985-01-01 12:57:18.722334
1

1985-01-01 18:18:21.603842
1985-01-01 16:18:39.602934
1985-01-02 18:43:43.838048
1985-01-02 22:12:35.583707
1985-01-01 17:16:17.508747
1985-01-01 17:14:38.402853
1985-01-01 15:14:43.940946
1985-01-02 20:48:52.568979
1985-01-02 01:13:30.724754
1985-01-01 23:34:30.215708
1985-01-02 16:55:29.286575
1985-01-02 08:50:59.350465
1985-01-02 08:03:45.322925
1985-01-02 16:53:21.606560
1985-01-02 13:24:46.806993
1985-01-03 14:58:53.051548
1985-01-02 03:29:16.805706
1985-01-02 03:39:55.640633
1985-01-02 10:37:01.205521
1985-01-02 17:58:37.927237
1985-01-03 01:31:12.009888
1985-01-03 12:49:34.638094
1985-01-03 06:57:05.515702
1985-01-01 16:18:39.602709
1985-01-01 17:10:20.256569
1985-01-01 17:43:26.402779
1985-01-01 13:55:14.982003
1985-01-03 05:23:26.409645
1985-01-03 11:27:29.529533
1985-01-03 05:53:45.606537
1985-01-03 00:39:40.808449
1985-01-02 13:05:47.122614
1985-01-01 22:29:21.123590
1985-01-01 13:46:12.002292
1985-01-01 09:15:34.201571
1985-01-02 20:50:24.007416
1985-01-02 00:11:10.130762
1

In [33]:
netCDF4.num2date(meanwh,u'days since 1985-01-01 00:00:00 UTC')

array([real_datetime(1985, 1, 3, 3, 53, 7, 209306),
       real_datetime(1985, 1, 1, 6, 59, 2, 401385),
       real_datetime(1985, 1, 3, 3, 56, 9, 602966), ...,
       real_datetime(1985, 1, 2, 10, 59, 31, 204917),
       real_datetime(1985, 1, 21, 19, 56, 38, 484389),
       real_datetime(1985, 1, 2, 4, 5, 5, 285073)], dtype=object)

In [24]:
#find all january wave heights. USE tt_, as it will then call all other variables
janWh = []
febWh = []
marWh = []
aprWh = []
mayWh = []
junWh = []
julWh = []
augWh = []
sepWh = []
octWh = []
novWh = []
decWh = []

for k in range(len(meanwh)):
    dt = netCDF4.num2date(meanwh[k],u'days since 1985-01-01 00:00:00 UTC')
    if dt.month == 1:
        janWh.append(k)
    if dt.month == 2:
        febWh.append(k)
    if dt.month == 3:
        marWh.append(k)
    if dt.month == 4:
        aprWh.append(k)
    if dt.month == 5:
        mayWh.append(k)
    if dt.month == 6:
        junWh.append(k)
    if dt.month == 7:
        julWh.append(k)
    if dt.month == 8:
        augWh.append(k)
    if dt.month == 9:
        sepWh.append(k)
    if dt.month == 10:
        octWh.append(k)
    if dt.month == 11:
        novWh.append(k)
    if dt.month == 12:
        decWh.append(k)

In [25]:
febWh

[]

In [30]:
janW = wh[janWh]
febW = wh[febWh]
marW = wh[marWh]
aprW = wh[aprWh]
mayW = wh[mayWh]
junW = wh[junWh]
julW = wh[julWh]
augW = wh[augWh]
sepW = wh[sepWh]
octW = wh[octWh]
novW = wh[novWh]
decW = wh[decWh]


print 'Month Min Max             Mean'
print 'Jan', janW.min(),janW.max(),janW.mean()
print 'Feb', febW.min(),febW.max(),febW.mean()
print 'Mar', marW.min(),marW.max(),marW.mean()
print 'Apr', aprW.min(),aprW.max(),aprW.mean()
print 'May', mayW.min(),mayW.max(),mayW.mean()
print 'Jun', junW.min(),junW.max(),junW.mean()
print 'Jul', julW.min(),julW.max(),julW.mean()
print 'Aug', augW.min(),augW.max(),augW.mean()
print 'Sep', sepW.min(),sepW.max(),sepW.mean()
print 'Oct', octW.min(),octW.max(),octW.mean()
print 'Nov', novW.min(),novW.max(),novW.mean()
print 'Dec', decW.min(),decW.max(),decW.mean()

 Month Min Max             Mean
Jan 0.25 5.050000190734863 1.6622956159023918
Feb

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
#why does this give me different mean wh value?

wh_ = wh[days]
qc_ = qc[days]
lon_ = lon[days]
tt_ = tt[days]
lat_ = lat[days]

print max(wh_)
print min(wh_)
print len(tt)
print len(days)
print np.mean(wh_)

In [ ]:
#for k in range(len(days)):
#    if wh_[k]>5:
#        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
#This should be average per track, wh above 7.

for k in range(len(wh_)):
    if wh_[k]>7:
        print wh_[k], netCDF4.num2date(tt_[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
daywh = [] #daystart
for k in range(len(tt)):
    if wh[k]<10:
        t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        if k == 0: #get day 0. Then else: every day after that
            daywh.append(0)
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            it = 0
        else:
            if t1.day != dd.day: #if day 1 is not equal to day 2, then append
                #print dd.day,t1.day
                daywh.append(k)
                it += 1
                dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
len(daywh)

In [ ]:
wh2 = wh[daywh]
qc2 = qc[daywh]
lon2 = lon[daywh]
tt2 = tt[daywh]
lat2 = lat[daywh]

print max(wh2)
print min(wh2)
print len(tt2)
print np.mean(wh2)

In [ ]:
for k in range(len(wh2)):
    if wh2[k]>7:
        print wh2[k], netCDF4.num2date(tt2[k],u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
netCDF4.num2date(12087.8427231626124,u'days since 1985-01-01 00:00:00 UTC')

In [ ]:
print np.mean(wh2), 'mean'
print max(wh2), 'max'
print min(wh2), 'min'

In [ ]:
 for p in range(len(timing)):
            #print(u,' - ',k,'/',len(urlON),' - ',p,'/',len(timing))
            if timing[p] >= start_date and timing[p] <= end_date:
                if lats[p]>latmin and lats[p]<latmax and lons[p]>lonmin and lons[p]<lonmax:
                    if np.where(np.logical_and(0<wh[p]<10,qc[p]==1)):
                        boxLat.append(lats[p])